In [16]:
import numpy as np
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import FloatSlider, Layout
from IPython.display import display

## Paths

In [4]:
# CodeCarbon example
CC_emissions_csv = "/home/nikhil/projects/green_comp_neuro/codecarbon/examples/emissions.csv"
CC_cloud_csv = "/home/nikhil/projects/green_comp_neuro/codecarbon/codecarbon/data/cloud/impact.csv"

project_dir = "/home/nikhil/projects/green_comp_neuro/fmriprep/"
template_emissions_csv = f"{project_dir}scripts/emissions.csv"
custom_emissions_csv = f"{project_dir}scripts/demo_emissions.csv"

## Read data

In [5]:
## Known cloud regions
CC_cloud_df = pd.read_csv(CC_cloud_csv)
cloud_countries = CC_cloud_df["countryName"].unique()
print(f"Number of countries with available data: {len(cloud_countries)}")

## Sample fmriprep run
template_emissions_df = pd.read_csv(template_emissions_csv)


Number of countries with available data: 23


## Interactive

In [32]:
def f(x,y,z):
    print(x,y,z)
    return x,y,z

def update_dropdown_2(*args):
    dropdown_2.options = CC_cloud_df[(CC_cloud_df["countryName"]==dropdown_1.value)]["region"].values


# Widgets
style = {'description_width': 'initial'}

title_label = widgets.Label( value='### fMRIPrep Carbon Calculator ###')

dropdown_1 = widgets.Dropdown(options=cloud_countries, value="Canada",
                                description="Select your country",style=style)
dropdown_2 = widgets.Dropdown(options=["Select country first"],value="Select country first", 
                                description="Select your cloud/HPC region",style=style)

dropdown_1.observe(update_dropdown_2, 'value')

sample_size_slider = FloatSlider(min=0, max=1000, step=100, description="Select number of scans",style=style)



ui = widgets.VBox([dropdown_1, dropdown_2, sample_size_slider])
config_out = widgets.interactive_output(f, {'x':dropdown_1, 'y':dropdown_2, 'z':sample_size_slider});

# Add a button 
calculate_button = widgets.Button(
    description='Calculate',
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
output = widgets.Output()

@output.capture()
def on_button_clicked(b):
    # print("Button clicked.")    
    config = config_out.outputs[0]["text"]
    country_name, cloud_region, n_scans = config.split(" ")
    n_scans = int(float(str.strip(n_scans)))

    on_cloud = "Y"
    project_name = f"fMRIPrep proc, n_scans ={n_scans}"

    print(f"Country: {country_name}, cloud_region: {cloud_region}, n_scans = {n_scans}")

    country_code = CC_cloud_df[(CC_cloud_df["countryName"]==country_name)]["countryIsoCode"].values[0]
    cloud_provider = CC_cloud_df[(CC_cloud_df["countryName"]==country_name) & 
                                (CC_cloud_df["region"]==cloud_region)]["provider"].values[0]

    locale_cols = ["project_name","country_iso_code","country_name","region","cloud_provider","cloud_region","on_cloud"]

    demo_df = template_emissions_df.copy()

    demo_df.loc[0,locale_cols] = [project_name, country_code,country_name,cloud_region,cloud_provider,cloud_region,on_cloud]
    multiple_runs_emissions = np.tile(demo_df.values,(n_scans,1))

    demo_df = pd.DataFrame(columns=template_emissions_df.columns, data=multiple_runs_emissions)

    demo_df.to_csv(custom_emissions_csv,index=False)
    print(f"emissions csv saved at: {custom_emissions_csv}")

calculate_button.on_click(on_button_clicked)

display(title_label)
display(ui)
display(calculate_button, output)

Label(value='### fMRIPrep Carbon Calculator ###')

Button(button_style='info', description='Calculate', icon='check', style=ButtonStyle())

Output()

In [15]:
! python /home/nikhil/projects/green_comp_neuro/codecarbon/codecarbon/viz/carbonboard.py $custom_emissions_csv

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app 'carbonboard' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Jun/2022 17:41:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2022 17:41:51] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2022 17:41:51] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2022 17:41:51] "GET /_dash-component-suites/dash_core_components/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2022 17:41:51] "GET /_dash-component-suites/dash_core_components/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2022 17:41:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2022 17:41:51] "GET /_dash-component-suites/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2022 17:41:51] "GET /_dash-component-suites/dash_table/async-table.js HTTP/1.1" 200 -
127.0.0.